In [1]:
from ROOT import TH1D, TCanvas, TF1, std
from pyLCIO import EVENT, UTIL, IOIMPL, IMPL
#from UTIL import CellIDDecoder
import matplotlib.pyplot as plt
import numpy as np

Welcome to JupyROOT 6.22/00
Loading LCIO ROOT dictionaries ...


In [2]:
import math
import sys
import string


class CellIDDecoder:

    """ decoder for LCIO cellIDs """

    def __init__(self,encStr):
        self.encStr=encStr
        self.funs = {} 

        tokens = encStr.split(',')
        
        offset = 0
        
        for t in tokens:
        
         # print "token: " , t
        
          st = t.split(':')
        
          if len(st)==2:
            name = st[0]
            start = offset 
            width = int(st[1])
            offset += abs( width )
        
          elif len(st)==3:
            name = st[0]
            start = int(st[1]) 
            width = int(st[2])
            offset = start + abs( width )
        
        
          else:
            print ("unknown token:" , t)
        
          mask = int(0x0)
          for i in range(0,abs(width)):
            mask = mask | ( 0x1 << ( i + start) )
        
          setattr( CellIDDecoder , name , self.makefun( mask, start , width) )


    def makefun(self, mask,start,width):
      if( width > 0 ):
        return ( lambda ignore, cellID : (( mask & cellID) >> start )  )
      else:
        return ( lambda ignore, cellID : (~(( mask & cellID) >> start )  ^ 0xffffffff) )

In [9]:
def Elayer(fName, maxEvt, collection):
    reader = IOIMPL.LCFactory.getInstance().createLCReader()
    reader.open( fName )
    
    
    nEvt = 0

    for evt in reader:
        nEvt += 1
        if nEvt > maxEvt:
                break
        ecalBarrel = evt.getCollection(collection)
    
        cellIDString = ecalBarrel.getParameters().getStringVal("CellIDEncoding")
        decoder = CellIDDecoder( cellIDString ) 
        print (cellIDString)
        for hit in ecalBarrel:
            l = decoder.layer( hit.getCellID0() ) 
            e = hit.getEnergy() 
            p = hit.getPosition()
            print (l,e, p[0], p[1], p[2])   
    
    


In [16]:
def hpfo(fName, maxEvt, collection):
    reader = IOIMPL.LCFactory.getInstance().createLCReader()
    reader.open( fName )
    
    nEvt = 0
    
        
    for evt in reader:
        nEvt += 1
        if nEvt > maxEvt:
                break
        
        col = evt.getCollection(collection)
        cellIDString = "system:0:5,module:5:3,stave:8:4,tower:12:4,layer:16:6,wafer:22:6,slice:28:4,cellX:32:-16,cellY:48:-16"
        decoder = CellIDDecoder( cellIDString ) 
        
        for c in col:
            hits = c.getCalorimeterHits()
            for h in hits:
                print (h.getEnergy(), decoder.layer(h.getCellID0()))
     
  

In [17]:
hpfo("nom50GeV_noHCALPFO_REC.slcio", 1, "PandoraClusters" )

0.01285629067569971 4
0.03623606637120247 6
0.007710502948611975 6
0.0036196685396134853 0
0.029192807152867317 6
0.028119133785367012 7
0.024566330015659332 7
0.05600739270448685 7
0.004542271140962839 0
0.007136449683457613 0
0.021763574331998825 7
0.016877586022019386 7
0.04428230971097946 7
0.0060171945951879025 7
0.004690441768616438 7
0.005262617487460375 7
0.017435744404792786 8
0.02891402319073677 8
0.015373106114566326 8
0.014107619412243366 0
0.004249946679919958 0
0.027949150651693344 8
0.1156715676188469 8
0.027969595044851303 8
0.010129911825060844 8
0.01622917875647545 8
0.007959643378853798 8
0.01070763636380434 8
0.02203783206641674 8
0.009231486357748508 9
0.06555242091417313 9
0.03051970712840557 9
0.05173666402697563 9
0.17095915973186493 9
0.009960360825061798 9
0.005792261101305485 0
0.025569235906004906 0
0.004876387305557728 0
0.007380604278296232 9
0.09260635823011398 9
0.03978653624653816 9
0.016844436526298523 9
0.003586103906854987 9
0.03452493995428085 9
0.0

In [13]:
#Elayer("nom50GeV_noHCALPFO_REC.slcio", 1, "EcalBarrelCollectionRec" )